In [ ]:
import numpy as np
import osmnx as ox
import pandas as pd
from loguru import logger
from pathlib import Path
import matplotlib.pyplot as plt

temp_data_dir = Path("temp_data")

output_file_path = Path("data/cities.csv")
missing_population = {
    "Izbica": 1739,
    "Piszczac": 2825,
    "Jawornik Polski": 1177,
    "Kamionka": 1702,
    "Latowicz": 1355,
    "Maciejowice": 1410,
    "Siennica": 2308,
    "Podkowa Leśna": 3874,
    "Olsztyn": 168_212,
    "Grabów": 1003,
    "Bobrowniki": 1104,
    "Pruszcz": 2710,
    "Mieścisko": 2362,
    "Miasteczko Krajeńskie": 1080,
}

cities = ox.features_from_place("Poland", {"place": ["city", "town", "village"]})
cities = cities.to_crs(epsg="2180")

# cities = cities[["name", "place", "population", "geometry"]].reset_index(drop=False)
# cities = cities[cities["geometry"].geom_type == "Point"]

# cities["lon"] = cities["geometry"].x
# cities["lat"] = cities["geometry"].y
# cities = cities.drop(columns=["geometry", "element_type"])

# cities["population"] = cities["population"].fillna(float("nan"))

# areas = []
# error_count = 0
# for city_name in cities["name"]:
#     try:
#         city_features = ox.geocode_to_gdf(f"{city_name}, {COUNTRY}")
#     except Exception:
#         logger.warning(f"Cannot get data for: {city_name}")
#         areas.append(np.nan)
#         error_count += 1
#         continue

#     logger.info(f"Getting data for: {city_name}")
#     city_features = city_features.to_crs(epsg=6933)
#     areas.append(city_features.area[0] / 1e6)

# logger.info(f"Got data for {len(cities) - error_count} cities, there were {error_count} errors.")

# cities["AREA"] = areas
# cities.columns = cities.columns.str.upper()
# cities["POPULATION"] = cities["POPULATION"].astype(float)

# cities = cities.merge(
#     pd.DataFrame(MISSING_POPULATION.items(), columns=["NAME", "POPULATION"]),
#     on="NAME", how="left", suffixes=("", "_MISSING"),
# )
# cities["POPULATION"] = cities["POPULATION"].fillna(cities["POPULATION_MISSING"])
# cities["POPULATION"] = cities["POPULATION"].astype(int)
# cities = cities.drop(columns=["POPULATION_MISSING"])

# cities.to_csv(FILE_PATH, index=False)


In [ ]:
temp = cities.loc["node"]



In [ ]:
temp3 = cities.loc["node"]
temp3 = temp3[temp3["name"] == "Stężyca"]
temp3

In [ ]:
temp

In [ ]:
temp = cities.loc["relation", :]
temp = temp[temp["name"] == "Stężyca"]

In [ ]:
fig, ax = plt.subplots()

temp.plot(ax=ax, facecolor="none", edgecolor="red", linewidth=1)
temp3.iloc[2:3].plot(ax=ax)

In [ ]:
temp2 = cities.loc["way"]
temp2 = temp2[temp2["name"] == "Wojnowo"]
temp2.plot(facecolor="none", edgecolor="red", linewidth=1)